import 


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from pymoo.indicators.hv import Hypervolume

Individual

In [ ]:
class Individual:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.objectives = []
        self.dominated_by = set()
        self.dominates = 0
        self.rank = 0
df = pd.read_csv ('DS02.csv' , header=0)
df = df.reindex (np.random.permutation(df.index))
df = df.reset_index(drop=True)
def initialize_population(pop_size, chromosome_length):
    population = [Individual(np.random.randint(low=0 , high=2, size=chromosome_length)) for _ in range(pop_size)]
    return population

mutation

In [ ]:
def flip (x):
    if x ==0:
        return 1
    else:
        return 0
def mutation(ind, m_rate):
    m_chromosome = ind.chromosome.copy()
    for i in range(len(m_chromosome)):
        if np.random.rand() < m_rate:
            m_chromosome[i] = flip(m_chromosome[i])
    return Individual(m_chromosome)

crowding_distance

In [ ]:
def crowding_distance(front):
    front_size = len(front)
    dis = np.zeros(front_size)

    for m in range(len(front[0].objectives)):
        sorted_front = sorted(front, key=lambda ind: ind.objectives[m])
        dis[0] = dis[-1] = float('inf')

        for i in range(1, front_size - 1):
            dis[i] += sorted_front[i + 1].objectives[m] - sorted_front[i - 1].objectives[m]

    return sorted(zip(front, dis), key=lambda x: x[1], reverse=True)

selection

In [ ]:
def environmental_selection( pop , pop_size ):
    remaining_pop = []
    i = 0
    while len(remaining_pop) + len(pop[i]) <= pop_size:
        remaining_pop.extend(pop[i])
        i += 1
    if len(remaining_pop) < pop_size:
        sorted_front = crowding_distance (pop[i])
        remaining_pop.extend(sorted_front[j][0] for j in range(pop_size - len(remaining_pop)))

    return remaining_pop

non_dominated_sorting & dominate

In [ ]:
def non_dominated_sorting(pop):
    fronts = [[]]
    for ind in pop:
        ind.dominated_by = set()
        ind.dominates = 0
        for s in pop:
            if ind != s :
                if all(ind.objectives[i] <= s.objectives[i] for i in range(len(ind.objectives))):
                    ind.dominated_by.add(s)
                elif all(ind.objectives[i] >= s.objectives[i] for i in range(len(ind.objectives))):
                    ind.dominates += 1
        if ind.dominates == 0:
            ind.rank = 0
            fronts[0].append(ind)

    i = 0
    while fronts[i]:
        FN = []
        for ind in fronts[i]:
            for s in ind.dominated_by:
                s.dominates -= 1
                if s.dominates == 0:
                    s.rank = i + 1
                    FN.append(s)
        i += 1
        fronts.append(FN)
    fronts = fronts[:-1]
    return fronts

def dominate ( ind1 , ind2):
        return all (ind1.objectives[i]<=ind2.objectives[i] for i in range(len(ind1.objectives)))

Roulette_wheel_selection

In [ ]:
def Roulette_wheel_selection(p , Q):
    probability_sum = [sum(p[:i+1]) for i in range(Q)]
    rand = np.random.rand()
    for i, next in enumerate(probability_sum):
        if rand <= next:
            o_idx = i+1
            break
    
    return o_idx

cross_over

In [ ]:
def initOSP(Q):
        return [1/Q for i in range (Q)]
def one_point_crossover(p1, p2):
        child_1 , child_2 = None , None
        points = np.random.randint(low=0 , high=len(p1.chromosome)+1)
        child_1 = (p1.chromosome[0:points] + p2.chromosome[points: ])
        child_2 = (p2.chromosome[0:points] + p1.chromosome[points: ])
        child_1 , child_2 = Individual(child_1) , Individual(child_2)
        return child_1 , child_2
def two_point_crossover(p1 ,p2):
        child_1 , child_2 = None , None
        points = [[np.random.randint(low=0 , high=len(p1.chromosome)+1)] for x in range(2)]
        points.sort(key=lambda x:x)
        child_1 = (p1.chromosome[0:points[0][0]] + p2.chromosome[points[0][0]:points[1][0]]+ p1.chromosome[points[1][0]: ] )
        child_2 = (p2.chromosome[0:points[0][0]] + p1.chromosome[points[0][0]:points[1][0]]+ p2.chromosome[points[1][0]: ] )
        child_1 , child_2 = Individual(child_1) , Individual(child_2)
        return child_1 , child_2
    
def uniform_crossover( p1 , p2):


    child_1 , child_2 = [] , []
    for i in range (len(p1.chromosome)):
        if (np.random.random()<0.50):
            child_1.append(p1.chromosome[i])
            child_2.append(p2.chromosome[i])
        else:
            child_1.append(p2.chromosome[i])
            child_2.append(p1.chromosome[i])
    child_1 , child_2 = Individual(child_1) , Individual(child_2)
    return child_1  , child_2

def shuffle_crossover(p1, p2):

    lst_1 , lst_2 = p1.chromosome , p2.chromosome
    np.random.shuffle(lst_1)
    np.random.shuffle(lst_2)
    child_1 , child_2 = [] , []
    for i in range (len(lst_1)):
        if (np.random.random()<0.50):
            child_1.append(lst_1[i])
            child_2.append(lst_2[i])
        else:
            child_1.append(lst_2[i])
            child_2.append(lst_1[i])
    child_1 , child_2 = Individual(child_1) , Individual(child_2)
    return child_1 , child_2

def reduced_surrogate_crossover(p1, p2):
    tmp_list_1 = []
    for index in range(len(p1.chromosome)-1):
        if p1.chromosome[index] != p1.chromosome[index+1] and p2.chromosome[index] != p2.chromosome[index+1]:
            tmp_list_1.append(index+1)
    if (len(tmp_list_1) != 0):
        point = np.random.choice(tmp_list_1)
        child_1 = (p1.chromosome[0:point] + p2.chromosome[point: ])
        child_2 = (p2.chromosome[0:point] + p1.chromosome[point: ])
        child_1 , child_2 = Individual(child_1) , Individual(child_2)
    else:
        child_1 ,child_2 = Individual(p1.chromosome) , Individual(p2.chromosome)
    return child_1 , child_2

def Crossover(parents , idx , c_rate):
    if (np.random.rand()<c_rate):
        match idx:
            case 1:
                return one_point_crossover(p1=parents[0] , p2=parents[1])
            case 2:
                return two_point_crossover(p1=parents[0] , p2=parents[1])
            case 3:
                return uniform_crossover(p1=parents[0] , p2=parents[1])
            case 4:
                return shuffle_crossover(p1=parents[0] , p2=parents[1])
            case 5:
                return reduced_surrogate_crossover (p1=parents[0] , p2=parents[1])
            case default:
                return None
    else:
        return parents

Update_OSP

In [ ]:
def Update_OSP(RD , PN  , Q):
    
    lst = []
    for q in range (Q):
        S1_q = sum ([row[q] for row in RD])
        S2_q = sum ([row[q] for row in PN])
        if S1_q == 0: 
            S3_q = .0001
        else:
            S3_q = S1_q
        S4_q = S1_q / (S2_q + S3_q)
        lst.append(S4_q)
    P_q = [(p_q / sum(lst)) for p_q in lst]
    return P_q

fitness_evaluation

In [ ]:
def fitness_evaluation( ind ):
    fp = ind.chromosome
    cl = [ x for x in range(len(fp))  if fp[x]==1 ]
    cl.append(df.shape[1]-1)
    data = df.iloc[: , cl]
    folds = []
    folds.append(data.iloc[0:np.floor(len(data)/3).astype(int)])
    folds.append(data.iloc[np.floor(len(data)/3).astype(int):np.floor(2*len(data)/3).astype(int)])
    folds.append(data.iloc[np.floor(2*len(data)/3).astype(int):])
    train_v , test_v = [] , []
    train_v.append((pd.concat([folds[0] , folds[1]] , axis=0  )).reset_index(drop=True)) ; test_v.append((folds[2]).reset_index(drop=True))
    train_v.append((pd.concat([folds[0] , folds[2]] , axis=0  )).reset_index(drop=True)) ; test_v.append((folds[1]).reset_index(drop=True))
    train_v.append((pd.concat([folds[1] , folds[2]] , axis=0  )).reset_index(drop=True)) ; test_v.append((folds[0]).reset_index(drop=True))
    fold_result = []
    for X_train , X_test in zip (train_v ,test_v ):

        scaler  = StandardScaler()
        y_train = X_train.iloc[:,X_train.shape[1]-1:]
        y_train = (np.array(y_train)).reshape(-1)
        X_train = X_train.iloc[:,:-1]
        X_train = np.array(X_train)
        y_test  = X_test.iloc[:,X_test.shape[1]-1:]
        y_test  = (np.array(y_test)).reshape(-1)
        X_test  = X_test.iloc[:,:-1]
        X_test  = np.array(X_test)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)
        knn = KNeighborsClassifier (n_neighbors= 3)
        knn.fit(X_train , y_train)
        y_pred = knn.predict(X_test)
        wrong  = 0
        all_instances = len(X_train) + len(X_test)
        for p , y in zip (y_pred , np.array(y_test)):
            if p != y:
                wrong += 1
        fold_result.append((wrong/all_instances) * 100)
    f1 = np.average(fold_result)
    f2 = sum(ind.chromosome)
    return f1 , f2

uniform_mutation

In [ ]:
def uniform_mutation( p , m_rate ):
    lst = []
    for ind in p:
        tmp = []
        for i in range(len(ind.chromosome)):
            if(np.random.random() < m_rate ):
                tmp.append( int (ind.chromosome[i] ^ True) )
            else:
                tmp.append(ind.chromosome[i])
        lst.append(tmp)
    chromosome_list = []
    for i in lst:
        chromosome_list.append(Individual(i))
    return chromosome_list

dominanceComparison

In [ ]:
def dominanceComparison ( p ):

    p_dominated , p_non_dominated = [] , []
    for j in p:
        for ind in p:
            if j != ind:
                if all(j.objectives[i]<= ind.objectives[i] for i in range(len(j.objectives))):
                    p_non_dominated.append(j)
                elif all(j.objectives[i]>=ind.objectives[i] for i in range(len(j.objectives))):
                    p_dominated.append(j)
    return p_dominated , p_non_dominated

def CreditAssignment(P , R  , np , nr , operator_idx):
       
        nPenalty , nReward = np.copy() , nr.copy()
        for ind in R:
            if len(ind.objectives) == 0 :
                ind.objectives = fitness_evaluation(ind)
        P_d , P_nd = dominanceComparison(P)
        if len(P_d) != 0: .
            for i in range (2):
                if (dominate ( P_d[0] , R[i] )):
                    nPenalty[operator_idx-1] = nPenalty[operator_idx-1] + 1
                else:
                    nReward[operator_idx-1]  = nReward[operator_idx-1] + 1
        else: 
            for i in range (2):
                if (not dominate (P[0] , R[i]) and not dominate (P[1] , R[i])) :
                    nReward[operator_idx-1]  = nReward[operator_idx-1] + 1
                else :
                    nPenalty[operator_idx-1] = nPenalty[operator_idx-1] + 1

        return nReward , nPenalty

MOBGA_AOS

In [ ]:
def MOBGA_AOS( crossover_rate, mutation_rate , maxFEs , Q , N , D , LP ):
    k , nFE , p_new , f0 = 0 , 0 , [] , 0
    p = [Individual([np.random.randint(low=0 , high=2) for _ in range (D)]) for _ in range(N)]
    for i in p :
        i.objectives = fitness_evaluation(i)
    nReward , nPenalty  , RD , PN = [0 for _ in range(Q)] , [0 for _ in range(Q)] , np.zeros((LP , Q)) , np.zeros((LP , Q))
    while nFE < maxFEs:
        p_new = []
        for i in range((np.floor(N/2)).astype(int)):
            pv = initOSP(Q)
            idx = Roulette_wheel_selection(pv,Q)
            P_p = [p[np.random.randint(low=0 , high=len(p))] for _ in range(2)]
            P_c = Crossover(P_p ,idx=idx , c_rate=crossover_rate )
            P_c = uniform_mutation (P_c , mutation_rate) 
            for i in range(len(P_c)):
                while all(P_c[i].chromosome[j] == 0 for j in range (len(P_c[i].chromosome))):
                    P_c[i] = (uniform_mutation( [P_c[i]] , 0.1))[0]
            nFE += 2
            nReward , nPenalty =CreditAssignment(P_p , P_c , operator_idx=idx , np=nPenalty , nr=nReward)
            p_new.extend(P_c)
        k +=1
        RD [k-1] = nReward
        PN [k-1] = nPenalty
        if k == LP:
            pv = Update_OSP(RD , PN , Q) 
            k=0
        R = p + p_new 
        front = non_dominated_sorting (R)
        PF = environmental_selection(front , N )
        p = PF
        f0  = front[0]
    optimal_feature_subset = PF
    return optimal_feature_subset  , f0


c_rate = 0.9
D = len(df.columns)-1
maxFEs = 3000
pop_size = 100
number_of_objectives = 2
number_of_CrossOver_function = 5
LP = 5
m_rate = 1/D
optimal_subset  , front_pareto = MOBGA_AOS (crossover_rate=c_rate , mutation_rate=m_rate,maxFEs=maxFEs , Q=number_of_CrossOver_function , N=pop_size , D=D , LP=LP)
x = [x.objectives[0] for x in front_pareto]
y = [x.objectives[1] for x in front_pareto]
plt.scatter(y, x , marker='o')
plt.xlabel('Solution size')
plt.ylabel('Classification Error')
plt.show()

mean = np.mean(hv)
std = np.std(hv)
x = [x.objectives[0] for x in front_pareto]
y = [x.objectives[1] for x in front_pareto]
plt.scatter(y, x , marker='o')
plt.xlabel('Solution size')
plt.ylabel('Classification Error')
plt.show()